In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
# import ikn.csv
ikn = pd.read_csv('ikn.csv', sep=';')
ikn.head()

,Tanggal,Username,Clean Dataset
0,2022-02-27 12:00:01+00:00,HIMASIERA,himbrenews halo sobat siera minggu enak baca b...
1,2022-02-27 09:23:52+00:00,oncombandung,bangun bedeng kali jagain lahan proyek btw bet...
2,2022-02-27 08:22:53+00:00,fadjroeL,temu dubes nur sultan kazakhstan nama ibu kota...
3,2022-02-27 04:04:47+00:00,Maandala_06,ikn indonesia resmi pindah dki jakarta nusanta...
4,2022-02-27 03:09:00+00:00,tribunkaltim,kepala otorita ikn tunggu umum jokowi struktur...


In [3]:

# Read stopwords from file
with open('stopwords.txt', 'r') as stop_file:
    stopwords = stop_file.read().split('\n')

# Check for NaN values and replace them with an empty string
ikn['Clean Dataset'] = ikn['Clean Dataset'].replace(np.nan, '', regex=True)

# Remove non-letter characters
ikn['Clean Dataset'] = ikn['Clean Dataset'].str.replace('[^a-zA-Z]', ' ')

# Remove stopwords
ikn['Clean Dataset'] = ikn['Clean Dataset'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stopwords]))

# Print the result
ikn.head()


,Tanggal,Username,Clean Dataset
0,2022-02-27 12:00:01+00:00,HIMASIERA,himbrenews halo sobat siera minggu enak baca b...
1,2022-02-27 09:23:52+00:00,oncombandung,bangun bedeng kali jagain lahan proyek btw bet...
2,2022-02-27 08:22:53+00:00,fadjroeL,temu dubes nur sultan kazakhstan nama kota nus...
3,2022-02-27 04:04:47+00:00,Maandala_06,ikn indonesia resmi pindah dki jakarta nusanta...
4,2022-02-27 03:09:00+00:00,tribunkaltim,kepala otorita ikn tunggu jokowi struktur bada...


In [17]:
import nltk

# Download the vader_lexicon resource
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to C:\Users\GLOBAL
[nltk_data]     KOMPUTER\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json
import reprlib

# Initialize SentimentIntensityAnalyzers
sia1A, sia1B, sia2 = SentimentIntensityAnalyzer(), SentimentIntensityAnalyzer(), SentimentIntensityAnalyzer()
sia1A.lexicon.clear()
sia1B.lexicon.clear()
sia2.lexicon.clear()

# Load custom lexicons
with open('leksikon/inset/_json_inset-neg.txt') as f:
    data1A = f.read()
with open('leksikon/inset/_json_inset-pos.txt') as f:
    data1B = f.read()
with open('leksikon/sentistrength_id/_json_sentiwords_id.txt') as f:
    data2 = f.read()

insetNeg = json.loads(data1A)
insetPos = json.loads(data1B)
senti = json.loads(data2)

# Update lexicons
sia1A.lexicon.update(insetNeg)
sia1B.lexicon.update(insetPos)
sia2.lexicon.update(senti)

# Apply sentiment analysis to the 'Clean Dataset' column
ikn['Sentiment1A'] = ikn['Clean Dataset'].apply(lambda x: sia1A.polarity_scores(x)['compound'])
ikn['Sentiment1B'] = ikn['Clean Dataset'].apply(lambda x: sia1B.polarity_scores(x)['compound'])
ikn['Sentiment2'] = ikn['Clean Dataset'].apply(lambda x: sia2.polarity_scores(x)['compound'])

# Map compound scores to labels (positive, neutral, negative)
ikn['Label1A'] = ikn['Sentiment1A'].apply(lambda x: 'positive' if x > 0 else ('neutral' if x == 0 else 'negative'))
ikn['Label1B'] = ikn['Sentiment1B'].apply(lambda x: 'positive' if x > 0 else ('neutral' if x == 0 else 'negative'))
ikn['Label2'] = ikn['Sentiment2'].apply(lambda x: 'positive' if x > 0 else ('neutral' if x == 0 else 'negative'))


In [5]:
# Print the result
ikn.head()

,Tanggal,Username,Clean Dataset,Sentiment1A,Sentiment1B,Sentiment2,Label1A,Label1B,Label2
0,2022-02-27 12:00:01+00:00,HIMASIERA,himbrenews halo sobat siera minggu enak baca b...,-0.9849,0.9776,0.7184,negative,positive,positive
1,2022-02-27 09:23:52+00:00,oncombandung,bangun bedeng kali jagain lahan proyek btw bet...,-0.9186,0.9325,0.0000,negative,positive,neutral
2,2022-02-27 08:22:53+00:00,fadjroeL,temu dubes nur sultan kazakhstan nama kota nus...,-0.8750,0.9750,0.7184,negative,positive,positive
3,2022-02-27 04:04:47+00:00,Maandala_06,ikn indonesia resmi pindah dki jakarta nusanta...,-0.9719,0.9682,0.8750,negative,positive,positive
4,2022-02-27 03:09:00+00:00,tribunkaltim,kepala otorita ikn tunggu jokowi struktur bada...,-0.9517,0.7906,0.0000,negative,positive,neutral


In [6]:
sample = "jokowi kok ada ada saja masa negara banyak hutang pengen bangun ibukota sih"
print("insetNeg: ", sia1A.polarity_scores(sample))
print("insetPos: ", sia1B.polarity_scores(sample))
print("insetCpdSum: 'compound':", sia1A.polarity_scores(sample)["compound"] + sia1B.polarity_scores(sample)["compound"])

print("senti\t: ", sia2.polarity_scores(sample))

insetNeg:  {'neg': 0.781, 'neu': 0.219, 'pos': 0.0, 'compound': -0.9799}
insetPos:  {'neg': 0.0, 'neu': 0.333, 'pos': 0.667, 'compound': 0.9638}
insetCpdSum: 'compound': -0.016100000000000003
senti	:  {'neg': 0.2, 'neu': 0.8, 'pos': 0.0, 'compound': -0.4588}


In [23]:
def is_positive_inset(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia1A.polarity_scores(tweet)["compound"] + sia1B.polarity_scores(tweet)["compound"] > 0

def is_positive_senti(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia2.polarity_scores(tweet)["compound"] > 0

In [7]:
# show 20 data ikn
ikn.head(20)



,Tanggal,Username,Clean Dataset,Sentiment1A,Sentiment1B,Sentiment2,Label1A,Label1B,Label2
0,2022-02-27 12:00:01+00:00,HIMASIERA,himbrenews halo sobat siera minggu enak baca b...,-0.9849,0.9776,0.7184,negative,positive,positive
1,2022-02-27 09:23:52+00:00,oncombandung,bangun bedeng kali jagain lahan proyek btw bet...,-0.9186,0.9325,0.0000,negative,positive,neutral
2,2022-02-27 08:22:53+00:00,fadjroeL,temu dubes nur sultan kazakhstan nama kota nus...,-0.8750,0.9750,0.7184,negative,positive,positive
3,2022-02-27 04:04:47+00:00,Maandala_06,ikn indonesia resmi pindah dki jakarta nusanta...,-0.9719,0.9682,0.8750,negative,positive,positive
4,2022-02-27 03:09:00+00:00,tribunkaltim,kepala otorita ikn tunggu jokowi struktur bada...,-0.9517,0.7906,0.0000,negative,positive,neutral
5,2022-02-26 23:54:22+00:00,sandra_hut,ikn indonesia resmi pindah dki jakarta nusanta...,-0.9719,0.9682,0.8750,negative,positive,positive
6,2022-02-26 18:51:45+00:00,Bamus_betawi,pasca ikn bamus betawi aju nama jakarta tokoh ...,-0.6124,0.0000,0.0000,negative,neutral,neutral
7,2022-02-26 15:45:20+00:00,idkatacom,ma ruf amin presiden jokowi kantong nama calon...,-0.6124,0.8750,0.0000,negative,positive,neutral
8,2022-02-25 19:46:55+00:00,DewawarmanDayac,bukti bhwa smua sprti sulap ikn nasib kejar fu...,-0.9432,0.0000,-0.2500,negative,neutral,negative
9,2022-02-25 12:36:41+00:00,kalap_corona,ndak niat narasi jin buang anak lokasi ikn jer...,-0.9849,0.0000,-0.6124,negative,neutral,negative


In [17]:
# change all categorical data to numerical data
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
ikn['Label1A'] = labelencoder.fit_transform(ikn['Label1A'])
ikn['Label1B'] = labelencoder.fit_transform(ikn['Label1B'])
ikn['Label2'] = labelencoder.fit_transform(ikn['Label2'])
ikn.head(20)

,Tanggal,Username,Clean Dataset,Sentiment1A,Sentiment1B,Sentiment2,Label1A,Label1B,Label2
0,2022-02-27 12:00:01+00:00,HIMASIERA,himbrenews halo sobat siera minggu enak baca b...,-0.9849,0.9776,0.7184,0,1,2
1,2022-02-27 09:23:52+00:00,oncombandung,bangun bedeng kali jagain lahan proyek btw bet...,-0.9186,0.9325,0.0000,0,1,1
2,2022-02-27 08:22:53+00:00,fadjroeL,temu dubes nur sultan kazakhstan nama kota nus...,-0.8750,0.9750,0.7184,0,1,2
3,2022-02-27 04:04:47+00:00,Maandala_06,ikn indonesia resmi pindah dki jakarta nusanta...,-0.9719,0.9682,0.8750,0,1,2
4,2022-02-27 03:09:00+00:00,tribunkaltim,kepala otorita ikn tunggu jokowi struktur bada...,-0.9517,0.7906,0.0000,0,1,1
5,2022-02-26 23:54:22+00:00,sandra_hut,ikn indonesia resmi pindah dki jakarta nusanta...,-0.9719,0.9682,0.8750,0,1,2
6,2022-02-26 18:51:45+00:00,Bamus_betawi,pasca ikn bamus betawi aju nama jakarta tokoh ...,-0.6124,0.0000,0.0000,0,0,1
7,2022-02-26 15:45:20+00:00,idkatacom,ma ruf amin presiden jokowi kantong nama calon...,-0.6124,0.8750,0.0000,0,1,1
8,2022-02-25 19:46:55+00:00,DewawarmanDayac,bukti bhwa smua sprti sulap ikn nasib kejar fu...,-0.9432,0.0000,-0.2500,0,0,0
9,2022-02-25 12:36:41+00:00,kalap_corona,ndak niat narasi jin buang anak lokasi ikn jer...,-0.9849,0.0000,-0.6124,0,0,0


In [21]:
from sklearn.feature_selection import SelectKBest, f_classif

X = ikn[['Sentiment1A', 'Sentiment1B', 'Sentiment2','Label1A', 'Label1B']]
y = ikn['Label2']  

selector = SelectKBest(score_func=f_classif, k=3)
X_new = selector.fit_transform(X, y)

selected_features = X.columns[selector.get_support()]
# TAMPILKAN NILAI MASING MASING

Sentiment1A	: Sentiment1A
Sentiment1B	: Sentiment1B
Sentiment2	: Sentiment2


In [22]:
# Menggunakan KNN dengan kernel linear sebagai classifier
# random_state merupakan pseudo randomization, supaya hasil yang didapat akan tetap sama setiap kali eksperimen diulang
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

X = ikn[['Sentiment1A', 'Sentiment1B', 'Sentiment2']]
y = ikn['Label2']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)    



KNeighborsClassifier()

In [23]:

# Memprediksi label dari data test
y_pred = classifier.predict(X_test)

In [24]:

# Membuat confusion matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[160   3   0]
 [  0 402   0]
 [  0   1 298]]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       163
           1       0.99      1.00      1.00       402
           2       1.00      1.00      1.00       299

    accuracy                           1.00       864
   macro avg       1.00      0.99      0.99       864
weighted avg       1.00      1.00      1.00       864



In [26]:
# validasi k fold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_score, recall_score, f1_score


# Membuat model KNN dengan k=5
classifier = KNeighborsClassifier(n_neighbors=5)

# Membuat objek KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# ...

# Membuat list untuk menampung hasil akurasi validasi
acc_scores = []
prec_scores = []
rec_scores = []
f1_scores = []

# Melakukan validasi silang
for train_index, test_index in kf.split(X):
    # Membagi data train dan data test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Melatih model dengan data train
    classifier.fit(X_train, y_train)

    # Memprediksi data test
    y_pred = classifier.predict(X_test)

    # Menghitung akurasi
    acc_score = classifier.score(X_test, y_test)
    acc_scores.append(acc_score)

    # Menghitung precision
    prec_score = precision_score(y_test, y_pred, average='macro')
    prec_scores.append(prec_score)

    # Menghitung recall
    rec_score = recall_score(y_test, y_pred, average='macro')
    rec_scores.append(rec_score)

    # Menghitung f1
    f1_sc = f1_score(y_test, y_pred, average='macro')  # Avoid using the same name
    f1_scores.append(f1_sc)

# ...


In [27]:

# Menampilkan hasil validasi silang
print('Akurasi: ', acc_scores)
print('Precision: ', prec_scores)
print('Recall: ', rec_scores)
print('F1 Score: ', f1_scores)


Akurasi:  [0.9953703703703703, 1.0, 0.9976825028968713, 0.9988412514484357, 0.9988412514484357]
Precision:  [0.9967159277504104, 1.0, 0.9984709480122325, 0.9992044550517104, 0.9992175273865415]
Recall:  [0.992750203473063, 1.0, 0.9967522143992733, 0.9987834549878345, 0.9987515605493135]
F1 Score:  [0.9946955124592846, 1.0, 0.9976056869803744, 0.9989923677682911, 0.9989829130868367]


In [28]:
# true positive (TP): jumlah data positif yang diprediksi benar
# true negative (TN): jumlah data negatif yang diprediksi benar
# false positive (FP): jumlah data negatif yang diprediksi salah
# false negative (FN): jumlah data positif yang diprediksi salah

# tampilkan tp tn fp fn

print("TP: ", confusion_matrix(y_test, y_pred)[0][0])
print("TN: ", confusion_matrix(y_test, y_pred)[1][1])
print("FP: ", confusion_matrix(y_test, y_pred)[0][1])
print("FN: ", confusion_matrix(y_test, y_pred)[1][0])

TP:  171
TN:  425
FP:  0
FN:  0
